In [1]:
import polars as pl
import re

In [67]:
data_df = pl.read_csv('/users/ujan/sports-language-in-politics/data/processed/politics_main_10m.csv').drop_nulls()

In [68]:
MILLISECONDS_IN_SECOND = 1000
datetimes = data_df.select((pl.col("created_utc") * MILLISECONDS_IN_SECOND).cast(pl.Datetime).dt.with_time_unit("ms").alias("datetime"))

In [69]:
data_df.replace("created_utc", datetimes['datetime'].dt.date())

/var/folders/by/rktr_w596p97pmt8_cbknvs80000gn/T/ipykernel_1316/2894318700.py:1: DeprecationWarning: `replace` is deprecated. DataFrame.replace is deprecated and will be removed in a future version. Please use
    df = df.with_columns(new_column.alias(column_name))
instead.
  data_df.replace("created_utc", datetimes['datetime'].dt.date())


id,created_utc,subreddit,category,super_category,body,author
str,date,str,str,str,str,str
"""hg3k6ci""",2021-10-10,"""eu""","""politics_2019""","""politics""","""&gt; Under the…","""delete013"""
"""esqrmpn""",2019-07-04,"""AOC""","""politics_2019""","""politics""","""Yet another de…","""CaptainAssPlun…"
"""fqhvq8d""",2020-05-13,"""AOC""","""politics_2019""","""politics""","""http://www.chi…","""RanDomino5"""
"""fq6h3em""",2020-05-10,"""AOC""","""politics_2019""","""politics""","""https://www.no…","""RanDomino5"""
"""ga2pgeh""",2020-10-25,"""AOC""","""politics_2019""","""politics""","""He has a hard …","""ChiraqBluline"""
"""f82hd5u""",2019-11-19,"""AOC""","""politics_2019""","""politics""","""Can we think a…","""balacio"""
"""g6pe38g""",2020-09-26,"""AOC""","""politics_2019""","""politics""","""Uh... Yes, *I’…","""noshoesyoulose…"
"""czdqjdg""",2016-01-27,"""MLS""","""soccer""","""sports""","""&gt; ""time-out…","""brakiri"""
"""g21rkdd""",2020-08-19,"""MLS""","""soccer""","""sports""","""Exactly. The p…","""smala017"""


In [70]:
df_15_21 = data_df.filter(pl.col('created_utc').dt.year().is_in([2015,2016,2017,2018,2019,2020,2021]))
df_15_21_oct = df_15_21.filter(pl.col('created_utc').dt.month().is_in([10]))

In [71]:
len(df_15_21_oct.filter((pl.col('subreddit') == 'The_Donald')))

55329

In [74]:
donald_df = df_15_21_oct.filter((pl.col('subreddit') == 'The_Donald'))

comments = [comment.replace("'", '') for comment in donald_df['body'].to_list()]
comments = [re.sub(r"[^a-zA-Z0-9]+", ' ', comment).lower() for comment in comments]

comments_long = []
    # filter by char
for c in range(len(comments)):
    if len(comments[c]) >= 50:
        comments_long.append(comments[c])
comments = comments_long

print('total comments : {}'.format(len(comments)))
word_count = 0
for comment in comments:
    word_count += len(comment.split())
print('total words : {}'.format(word_count))

total comments : 34212
total words : 1150026


In [73]:
len(df_15_21_oct.filter((pl.col('subreddit') == 'Conservative')))

10143

In [62]:
len(df_15_21_oct.filter((pl.col('subreddit') == 'AskTrumpSupporters')))

2605

In [64]:
len(df_15_21_oct.filter((pl.col('subreddit') == 'Donald_Trump')))

19

In [72]:
len(df_15_21_oct.filter((pl.col('subreddit') == 'politics')))

153547